<a href="https://colab.research.google.com/github/TimeIsAFlatLoop/Machine-Learning-/blob/main/lung_cancer_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import math
import numpy as np
import pandas as pd
import kagglehub
import os
import pickle

In [ ]:


# Download latest version
path = kagglehub.dataset_download("akashnath29/lung-cancer-dataset")

print("Path to dataset files:", path)

100%|██████████| 68.8k/68.8k [00:00<00:00, 50.9MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/akashnath29/lung-cancer-dataset/versions/2


In [ ]:


path = "/root/.cache/kagglehub/datasets/akashnath29/lung-cancer-dataset/versions/2"
print(os.listdir(path))

['dataset.json', 'dataset.csv']


In [ ]:
df = pd.read_csv(f"{path}/dataset.csv",encoding="utf-8")
print(df)

     GENDER  AGE  SMOKING  YELLOW_FINGERS  ANXIETY  PEER_PRESSURE  \
0         M   65        1               1        1              2   
1         F   55        1               2        2              1   
2         F   78        2               2        1              1   
3         M   60        2               1        1              1   
4         F   80        1               1        2              1   
...     ...  ...      ...             ...      ...            ...   
2995      F   71        2               1        1              2   
2996      F   75        1               2        1              1   
2997      F   62        2               2        2              1   
2998      M   30        1               1        2              2   
2999      M   40        1               2        2              1   

      CHRONIC_DISEASE  FATIGUE  ALLERGY  WHEEZING  ALCOHOL_CONSUMING  \
0                   2        1        2         2                  2   
1                   1      

In [ ]:

df.columns = df.columns.str.strip()  # Clean column names
df['LUNG_CANCER'] = (
    df['LUNG_CANCER']
    .astype(str)
    .str.upper()
    .str.strip()
    .map({'YES': 1, 'NO': 0})
)
df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,M,65,1,1,1,2,2,1,2,2,2,2,2,2,1,0
1,F,55,1,2,2,1,1,2,2,2,1,1,1,2,2,0
2,F,78,2,2,1,1,1,2,1,2,1,1,2,1,1,1
3,M,60,2,1,1,1,2,1,2,1,1,2,1,2,2,1
4,F,80,1,1,2,1,1,2,1,2,1,1,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,F,71,2,1,1,2,2,1,1,1,1,2,1,1,2,0
2996,F,75,1,2,1,1,1,2,2,2,2,1,1,2,1,0
2997,F,62,2,2,2,1,2,2,2,2,1,1,2,2,2,1
2998,M,30,1,1,2,2,2,2,2,2,2,1,2,1,2,1


In [ ]:
df['GENDER'] = df['GENDER'].map({'M': 1, 'F': 0})
df

,GENDER,AGE,SMOKING,YELLOW_FINGERS,ANXIETY,PEER_PRESSURE,CHRONIC_DISEASE,FATIGUE,ALLERGY,WHEEZING,ALCOHOL_CONSUMING,COUGHING,SHORTNESS_OF_BREATH,SWALLOWING_DIFFICULTY,CHEST_PAIN,LUNG_CANCER
0,1,65,1,1,1,2,2,1,2,2,2,2,2,2,1,0
1,0,55,1,2,2,1,1,2,2,2,1,1,1,2,2,0
2,0,78,2,2,1,1,1,2,1,2,1,1,2,1,1,1
3,1,60,2,1,1,1,2,1,2,1,1,2,1,2,2,1
4,0,80,1,1,2,1,1,2,1,2,1,1,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,71,2,1,1,2,2,1,1,1,1,2,1,1,2,0
2996,0,75,1,2,1,1,1,2,2,2,2,1,1,2,1,0
2997,0,62,2,2,2,1,2,2,2,2,1,1,2,2,2,1
2998,1,30,1,1,2,2,2,2,2,2,2,1,2,1,2,1


In [ ]:

print(df.isnull().sum())  # See missing values

# Drop or fill any remaining NaNs
df.dropna(inplace=True)

GENDER                   0
AGE                      0
SMOKING                  0
YELLOW_FINGERS           0
ANXIETY                  0
PEER_PRESSURE            0
CHRONIC_DISEASE          0
FATIGUE                  0
ALLERGY                  0
WHEEZING                 0
ALCOHOL_CONSUMING        0
COUGHING                 0
SHORTNESS_OF_BREATH      0
SWALLOWING_DIFFICULTY    0
CHEST_PAIN               0
LUNG_CANCER              0
dtype: int64


In [ ]:
df.isna().sum()

,0
GENDER,0
AGE,0
SMOKING,0
YELLOW_FINGERS,0
ANXIETY,0
PEER_PRESSURE,0
CHRONIC_DISEASE,0
FATIGUE,0
ALLERGY,0
WHEEZING,0


In [ ]:

x = df[['AGE', 'SMOKING', 'YELLOW_FINGERS', 'ANXIETY', 'PEER_PRESSURE',
        'CHRONIC_DISEASE', 'FATIGUE', 'ALLERGY', 'WHEEZING',
        'ALCOHOL_CONSUMING', 'COUGHING', 'SHORTNESS_OF_BREATH',
        'SWALLOWING_DIFFICULTY', 'CHEST_PAIN']].values.tolist()

y = df['LUNG_CANCER'].values.tolist()

In [ ]:

n = len(x)           # Number of data points
d = len(x[0])        # Number of features

weights = [0.1] * d  # Initialize weights for each feature
bias = 0.1
alpha = 0.001
iterations = 1000

for _ in range(iterations):
    cross_entropy = 0
    grad_w = [0.0] * d
    grad_b = 0.0

    for i in range(n):
        z = sum(w * x[i][j] for j, w in enumerate(weights)) + bias
        pred = 1 / (1 + math.exp(-z))
        pred = max(min(pred, 1 - 1e-15), 1e-15)
        error = pred - y[i]
        cross_entropy += - (y[i] * math.log(pred) + (1 - y[i]) * math.log(1 - pred)) / n
        for j in range(d):
            grad_w[j] += (x[i][j] * error) / n
        grad_b += error / n

    # Update weights and bias
    weights = [w - alpha * gw for w, gw in zip(weights, grad_w)]
    bias -= alpha * grad_b

print(f"Final weights: {weights}")
print(f"Final bias: {bias}")
print(f"Final loss: {cross_entropy}")

Final weights: [-0.021019154904387344, 0.0641456920509699, 0.06478673194900086, 0.059530663853587705, 0.05602531153946745, 0.0573782785810844, 0.06153560747427565, 0.06406060991079389, 0.051152714199832916, 0.05283907006043739, 0.06805955599716866, 0.06103737089210741, 0.058783402456041786, 0.061664179623600233]
Final bias: 0.07720049008659048
Final loss: 0.7020418960443053
